In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from torch.autograd import Variable

"""Change to the data folder"""
train_path = "./new_train/"
val_path = "./new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=train_path)
val_dataset =  ArgoverseDataset(data_path=val_path)

In [3]:
batch_sz = 32

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp =  torch.FloatTensor(inp)
    out =  torch.FloatTensor(out)
    
    return [inp, out]

In [4]:
batch_sz = 32

def my_collate2(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    #out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    a_id =[numpy.dstack([scene['agent_id']]) for scene in batch]
    t_id =[numpy.dstack([scene['track_id']]) for scene in batch]
    s_id =[numpy.dstack([scene['scene_idx']]) for scene in batch] 
    #inp = torch.LongTensor(inp)
    #out = torch.LongTensor(out)
    inp =  torch.FloatTensor(inp)
    #out =  torch.FloatTensor(out)
    
    return [inp,a_id,t_id,s_id]

In [5]:
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=1)
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate2, num_workers=1)

In [6]:
from torch import nn,optim
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel,self).__init__()
        self.hidden_dim = 2048
        self.num_layers = 3
        self.lstm = nn.LSTM(240,self.hidden_dim, num_layers= self.num_layers, batch_first = True)
        self.linear = nn.Conv1d(self.hidden_dim,240,1)

    def forward(self,x):
       #input : batch sze x  timesteps x 240
        x,(h,c) = self.lstm(x)

        #final hidden layer: batch_size x timesteps x hidden_dim
        x = x .transpose(1,2)

        #batch size x hidden_dim x timesteps
        x = self.linear(x)

        # last step to : project hidden_dim -> 240
        x = x.transpose(1,2)

        return x

    def forward_test(self, x, num_steps = 30):
        res = []

        #Hidden states initiallized to 0 
        h = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        c = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        
        # run input for first 19 steps, take hidden state at last time step
        # perform projection an add to list of outputs
        # want to use the output of the 19 time step to predict the output of the 20th
        for step in range((num_steps)):
            # output, [hidden_states]
            x,(h,c) = self.lstm(x,(h,c))
            #take the hidden state at the last time step 
            x = x[:,-1:]
            x = x.tranpose(1,2)
            x = self.linear(x)
            x = x.transpose(1,2)
            res.append(x)
        res = torch.cat(res,1)
        return res

In [7]:
# Throw all the input and model information into the gpu 
model = LSTMModel().cuda()   # will turn float tensors into cuda tensors

optimizer = optim.Adam(model.parameters(), lr= 1e-3)

# Exponentially Moving Average 
loss_ema = -1
loss_ema2 = -1

In [8]:
for epoch in range(20):
    # after a few epochs if nothing is improving lower your learning rate 
    for i_batch, sample_batch in enumerate(train_loader):
        
        model.zero_grad()
        optimizer.zero_grad()
        
        inp,out = sample_batch
        #print("Inp Size :{}".format(inp.shape))
        #print("Out Size :{}".format(out.shape))
        inp = inp.cuda()
        out = out.cuda()
        
        

        # COncatenate input and output along the timestep axis : batch_sz x 60 x 49 x 4
        mixed = torch.cat([inp,out],2).transpose(1,2).reshape((-1,49,240))#.float()
        
        #print("Mixed Size: {}".format(mixed.shape))

       # Feed every time step but the final time step into the model 
        
        #print("19 + 30 time steps:{}".format(mixed[:,:,-1].shape))
        #print("30 Time Steps Output: {}".format(mixed[:,:,-1][:,-30:].shape))
        
        time_step = mixed[:,:,:][:,-30:,:]
        #print("time step dims: {}".format(time_step.shape))
        
        #y_pred= model(mixed[:,:,-1])[:,-30:] # in: first 48 time steps , gives predictions timesteps 2-49
        
       
        
        
        y_pred = model(time_step)

        
        y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)
        #print(y_pred.shape)

        # compare prediction to actual output [Ground Truth]. Take the root mean squared error
        loss = (torch.mean((y_pred - out)**2))**0.5
       
        loss.backward()
        optimizer.step()
        
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema*0.99 + loss*0.01

        # Measure the difference between using the ground truth value and not using it[ Used for validation data ]
        #with torch.no_grad():
         #   y_pred2 = model.forward_test(inp.transpose(1,2). reshape((-1,19,240)))
          #  y_pred2 = y_pred2.reshape((-1,30,60,4)).transpose(1,2)
           # loss2 = torch.mean((y_pred2 - out)**2)**0.5

        
        # Every 10th iteration print out the loss and exponentially moving average 
        if i_batch %10 == 0:
            print("loss full", epoch, i_batch, loss_ema.item(),loss.item())
            #print("loss test", epoch, i_batch, loss_ema2.item(),loss2.item())
            
        #if i_batch > 3215:
         #   break
            


loss full 0 0 536.7509155273438 536.7509155273438
loss full 0 10 540.6055297851562 598.7528686523438
loss full 0 20 543.7275390625 535.7133178710938
loss full 0 30 545.1627807617188 551.1118774414062
loss full 0 40 548.0517578125 554.4989624023438
loss full 0 50 548.5067749023438 703.252685546875
loss full 0 60 550.3306884765625 554.0475463867188
loss full 0 70 552.067626953125 555.3121337890625
loss full 0 80 552.67236328125 508.220703125
loss full 0 90 551.4139404296875 576.3455200195312
loss full 0 100 549.1747436523438 464.3990783691406
loss full 0 110 547.7603759765625 523.2324829101562
loss full 0 120 544.0805053710938 574.144775390625
loss full 0 130 542.3776245117188 526.3751220703125
loss full 0 140 540.4844360351562 521.0901489257812
loss full 0 150 539.4710693359375 539.3063354492188
loss full 0 160 537.598876953125 531.108642578125
loss full 0 170 535.419921875 557.129150390625
loss full 0 180 532.5202026367188 512.94921875
loss full 0 190 530.0391235351562 534.443603515625

In [9]:
print(y_pred.shape)
print(out.shape)

torch.Size([22, 60, 30, 4])
torch.Size([22, 60, 30, 4])


In [10]:
#Find the index of an agent_id within the track_id of a current scence 
def find_index(track_id, agent_id):
    track_id = list(track_id)
    index = track_id.index(agent_id)
    return index

In [29]:
#   for epoch in range(20):

# y = ["v1", "v2", "v3", "v4", "v5", "v6", "v7", "v8", "v9", "v10", "v11", "v12", "v13", "v14", "v15", "v16", "v17", "v18", "v19", "v20", "v21", "v22", "v23", "v24", "v25", "v26", "v27", "v28", "v29", "v30", "v31", "v32", "v33", "v34", "v35", "v36", "v37", "v38", "v39", "v40", "v41", "v42", "v43", "v44", "v45", "v46", "v47", "v48", "v49", "v50", "v51", "v52", "v53", "v54", "v55", "v56", "v57", "v58", "v59", "v60"]
final_result = []
for i_batch , sample_batch in enumerate(val_loader):
    inp,a_id,t_id,s_id = sample_batch
    
    s_id = list(numpy.concatenate(s_id).flat)
    a_id = list(numpy.concatenate(a_id).flat)
    t_id = numpy.array(t_id)
    
    index_list_per_batch = []
    
    #Generate the indexes we need for each agent row 
    for i in range(32):
        flat_t_id = numpy.array(t_id[i])
        flat_t_id = flat_t_id[:,0,0]
        current_agent_id = a_id[i]
        
        
        index = find_index(flat_t_id, current_agent_id)
        
        index_list_per_batch.append(index)
    

    inp = inp.cuda()
    out =  torch.zeros(32,60,30,4)
    out = out.cuda()
    #print(inp.shape)
    #print(out.shape)
    mixed = torch.cat([inp,out],2).transpose(1,2).reshape((-1,49,240))
    #Use the model to ouput predicted positions for each agent in the 32 scenes
    #print("Inp: {}".format(inp.shape))
    #inp =  inp.transpose(1,2).reshape((-1,19,240))
   
   
    
    # Flatten tensor to the timestep dimension
    time_step = mixed[:,:,:][:,-30:,:]
    #time_step = inp[:,:,:][:,:,:]
    #print("Time Step Dim: {}".format(time_step.shape))
    
   
    
    y_pred = model(time_step)
    y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)
    #y_pred = model(time_step).reshape((-1,30,60,4))

    #print("Model Output : {}".format(y_pred.shape))
    
    #y_pred = y_pred
    #print("Reshape and Transpose {}".format(y_pred.shape))
    
    
    
    #y_pred = model(inp.reshape((len(inp),-1))).reshape((-1,60,30,4))
    
    
    # Retrive only the positions 
    y_pred = y_pred[:,:,:,[0,1]]
    
    #print("Only Positions : {}".format(y_pred.shape))
    for i in range(32):
        row_result = y_pred[i,index_list_per_batch[i],:,:]
        
        
        row_result = torch.flatten(row_result)
         
        
        
        row_result = row_result.cpu()
        row_result = row_result.detach().numpy()
        row_result = numpy.hstack(([s_id[i]], row_result))
        final_result.append(row_result)
        
        
        
    
    
    
    
    
    
   
     

In [30]:
#import pandas as pd 
#print(final_result)
#y = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', "v7", "v8", "v9", "v10", "v11", "v12", "v13", "v14", "v15", "v16", "v17", "v18", "v19", "v20", "v21", "v22", "v23", "v24", "v25", "v26", "v27", "v28", "v29", "v30", "v31", "v32", "v33", "v34", "v35", "v36", "v37", "v38", "v39", "v40", "v41", "v42", "v43", "v44", "v45", "v46", "v47", "v48", "v49", "v50", "v51", "v52", "v53", "v54", "v55", "v56", "v57", "v58", "v59", "v60"]
#y = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60']
#final_result.insert(0, numpy.array(y))
final = numpy.array(final_result)
# var = numpy.stack((y, final))
print(final)
#final_df = pd.DataFrame(final,index=None,columns=y)
#final_df.columns = y 
#final_df
#final_df.to_csv("submission.csv")
#final_df = pd.DataFrame(final).to_csv("submission.csv", header=None, index=None)
#final_df.columns = y
numpy.savetxt("submission.csv", final, delimiter=",",fmt='%s')


[[10002.           542.94250488  -113.52547455 ...   172.08305359
    378.95880127   172.01843262]
 [10015.           304.18145752  -156.06607056 ...   -39.51491928
    155.08166504   -39.59290695]
 [10019.           542.94250488  -113.52547455 ...   172.08305359
    378.95880127   172.01843262]
 ...
 [ 9905.           542.94250488  -113.52547455 ...   172.08305359
    378.95880127   172.01843262]
 [ 9910.           905.96002197   101.18159485 ...   302.47918701
    778.43688965   302.48291016]
 [ 9918.           208.44741821   -45.61751556 ...   -38.24898529
    -83.34333801   -38.24599457]]
